In [28]:
import pandas as pd

import os
BASEDIR = os.getcwd()
df = pd.read_csv(os.path.join(BASEDIR,"app","static","uploads","20230602-183306_washingmachine.csv"),sep=';')
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')
monthly_sum = df['Wh'].resample('M').sum()
monthly_sum.index = monthly_sum.index.strftime('%b')

labels = monthly_sum.index.to_list()
values = monthly_sum.values.tolist()

# labels
# values


[101000,
 85300,
 96000,
 90500,
 96000,
 20000,
 20000,
 16000,
 16000,
 16000,
 16000,
 20000]

In [34]:
round(monthly_sum.values.mean(),2)

49400.0

In [36]:
round(monthly_sum.values.sum()*0.008/12,2)

395.2

# data

In [6]:
DATA_PATH = 'C:\\Users\\stefa\\OneDrive - FHWN\\Privat\\Studium\\MIT_2-Semester\\case_study\\lession1\\datasets\\ukdale2.h5'
from nilmtk import DataSet

ukdale = DataSet(DATA_PATH)
ukdale.set_window(start='2014-04-01', end='2014-07-01')

aggregate = next(ukdale.buildings[1].elec.mains().load(sample_period=10))
aggregate

2023-06-04 13:04:58,747:DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.
2023-06-04 13:04:58,751:DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.
2023-06-04 13:04:58,862:INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-06-04 13:04:58,863:INFO:numexpr.utils:NumExpr defaulting to 8 threads.


physical_quantity,power,voltage,power
type,apparent,,active
2014-04-01 00:00:00+01:00,375.773010,245.613998,293.061005
2014-04-01 00:00:10+01:00,375.470001,245.688995,292.940002
2014-04-01 00:00:20+01:00,373.877014,245.649994,291.472015
2014-04-01 00:00:30+01:00,377.635986,245.570007,294.808990
2014-04-01 00:00:40+01:00,374.385986,245.610992,291.983002
...,...,...,...
2014-06-30 23:59:10+01:00,154.173004,243.438995,104.052002
2014-06-30 23:59:20+01:00,154.535004,243.289001,104.334000
2014-06-30 23:59:30+01:00,154.615005,243.356995,104.350998


In [36]:
import nilmtk
import os
import numpy as np

building = 5
appliance = 'microwave'
start_time = '2014-10-01'
end_time = '2014-10-02'
sample_period = 10



ukdale = nilmtk.DataSet(DATA_PATH)
ukdale.set_window(start=start_time, end=end_time)
mains = ukdale.buildings[building].elec.mains().power_series_all_data(sample_period=sample_period, resample_kwargs={'label': 'right'})


c:\users\stefa\onedrive - fhwn\privat\studium\mit_2-semester\case_study\smart_meter_predictions\nilmtk\nilmtk\utils.py:499: UserWarning: Not all resample_kwargs were consumed: {'label': 'right'}
  warnings.warn("Not all resample_kwargs were consumed: {}".format(repr(all_resample_kwargs)))


In [37]:
CSVPATH = 'app\\static\\uploads\\mains.csv'
# mains.to_csv('app\\static\\uploads\\mains.csv')
type(mains)
mains

2014-10-01 00:00:00+01:00    631.391968
2014-10-01 00:00:10+01:00    634.210999
2014-10-01 00:00:20+01:00    631.526978
2014-10-01 00:00:30+01:00    637.171997
2014-10-01 00:00:40+01:00    634.783020
                                ...    
2014-10-01 23:59:10+01:00    308.125000
2014-10-01 23:59:20+01:00    307.385010
2014-10-01 23:59:30+01:00    310.351013
2014-10-01 23:59:40+01:00    308.066010
2014-10-01 23:59:50+01:00    307.515015
Freq: 10S, Name: (power, active), Length: 8640, dtype: float32

In [45]:
import pandas as pd

mains = pd.read_csv(CSVPATH,header=None).rename(columns={0:'Date',1:'Power'})
mains['Date'] = pd.to_datetime(mains['Date'])
mains = mains.set_index('Date').iloc[:,0]
# type(mains)
mains


Date
2014-10-01 00:00:00+01:00    631.39197
2014-10-01 00:00:10+01:00    634.21100
2014-10-01 00:00:20+01:00    631.52700
2014-10-01 00:00:30+01:00    637.17200
2014-10-01 00:00:40+01:00    634.78300
                               ...    
2014-10-01 23:59:10+01:00    308.12500
2014-10-01 23:59:20+01:00    307.38500
2014-10-01 23:59:30+01:00    310.35100
2014-10-01 23:59:40+01:00    308.06600
2014-10-01 23:59:50+01:00    307.51500
Name: Power, Length: 8640, dtype: float64

In [50]:
def pre_process_input(data, window_size):
    half_window = window_size // 2
    data = np.array(data).reshape(-1, 1)
    data_processed = np.zeros((len(data) - window_size + 1, window_size, 1)) # Hinzufügen einer Dimension
    for i in range(len(data) - window_size + 1):
        data_processed[i] = data[i:i + window_size]
    return data_processed


window_size = 99 # ist im seq2point.py der wert sequence_lenght (default)
test_data_processed = pre_process_input(mains, window_size)

In [48]:
# test_data_processed

In [59]:
# !pip install git+https://github.com/nilmtk/nilm_metadata.git
# !pip install git+https://github.com/BHafsa/nilmtk

In [58]:
# TensorFlow and tf.keras
import tensorflow as tf
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

AttributeError: partially initialized module 'charset_normalizer' has no attribute 'md__mypyc' (most likely due to a circular import)

In [52]:
# from tensorflow.keras.layers import MaxPool1D, Activation, Layer, Conv1D, Dense, Dropout, Reshape, Flatten, Add, \
#     MaxPool1D, BatchNormalization, ZeroPadding1D
# import tensorflow as tf
# from tensorflow.keras.models import Sequential

# def return_seq2seq():
#     model = Sequential()
#     model.add(Conv1D(30, 10, activation="relu", input_shape=(99, 1), strides=2))
#     model.add(Conv1D(30, 8, activation='relu', strides=2))
#     model.add(Conv1D(40, 6, activation='relu', strides=1))
#     model.add(Conv1D(50, 5, activation='relu', strides=1))
#     model.add(Dropout(.2))
#     model.add(Conv1D(50, 5, activation='relu', strides=1))
#     model.add(Dropout(.2))
#     model.add(Flatten())
#     model.add(Dense(1024, activation='relu'))
#     model.add(Dropout(.2))
#     model.add(Dense(99))
#     optim = tf.keras.optimizers.Adam(learning_rate=1e-4)
#     model.compile(loss='mse', optimizer=optim)

#     return model

In [56]:
from tensorflow.keras.models import load_model

model_path = os.path.join(f'app\\weights\\seq2seq-temp-weights-microwave-epoch0.h5')
# model = load_model(model_path)

# predictions = model.predict(test_data_processed)
# predictions_inverse_scaled = scaler.inverse_transform(predictions)
# predictions

AttributeError: partially initialized module 'charset_normalizer' has no attribute 'md__mypyc' (most likely due to a circular import)